Read data.

In [1]:
import pandas as pd
import numpy as np
traindf = pd.read_csv('train.csv')
input_datadf = pd.read_csv('test.csv')

In [2]:
train_seq,input_seq = [],[]

for i in range(len(traindf)):
    train_seq.append(map(long,traindf['Sequence'].loc[i].split(',')))
for i in range(len(input_datadf)):
    input_seq.append(map(long,input_datadf['Sequence'].loc[i].split(',')))

        
print(len(train_seq),len(input_seq))
print(train_seq[0])
print(input_seq[0])


(113845, 113845)
[1L, 3L, 13L, 87L, 1053L, 28576L, 2141733L, 508147108L, 402135275365L, 1073376057490373L, 9700385489355970183L, 298434346895322960005291L, 31479360095907908092817694945L, 11474377948948020660089085281068730L]
[1L, 0L, 0L, 2L, 24L, 552L, 21280L, 103760L, 70299264L, 5792853248L, 587159944704L]


Shift sequence until length of sequence = 3.

In [24]:
train_shift, input_shift = [], []
for i in range(len(train_seq)):
    for j in range(1, len(train_seq[i])-3):
        train_shift.append(train_seq[i][j:])
for i in range(len(input_seq)):
    for j in range(1, len(input_seq[i])-3):
        input_shift.append(input_seq[i][j:])

In [25]:
print(len(train_shift),len(input_shift))

(4288798, 4160789)


Here I us marisa_trie to construct a trie. Find signatures of train/input sequences and their shifted versions. Then count the occurrences for each unique signature.

These two functions code are from Balzac.

https://www.kaggle.com/balzac/integer-sequence-learning/prefixes-lookup-0-22/discussion

In [3]:
import marisa_trie
import fractions
def findGCD(seq):
    gcd = seq[0]
    for i in range(1,len(seq)):
        gcd = fractions.gcd(gcd,seq[i])
    return abs(gcd)

def findSignature(seq):
    nonzero_seq = [d for d in seq if d!=0]
    if len(nonzero_seq)==0:
        return seq, 0
    sign = 1 if nonzero_seq[0]>0 else -1
    gcd = findGCD(seq)
    return [sign*x//gcd for x in seq], sign*gcd

In [26]:
train_str_seq,input_str_seq = [], []
train_shift_str_seq,input_shift_str_seq = [], []

for i in range(len(train_seq)):
    signature, _ = findSignature(train_seq[i])
    train_str_seq.append(','.join(map(str, signature)))
for i in range(len(input_seq)):
    signature, _ = findSignature(input_seq[i])
    input_str_seq.append(','.join(map(str, signature)))
for i in range(len(train_shift)):
    signature, _ = findSignature(train_shift[i])
    train_shift_str_seq.append(','.join(map(str, signature)))
for i in range(len(input_shift)):
    signature, _ = findSignature(input_shift[i])
    input_shift_str_seq.append(','.join(map(str, signature)))
    
def Construct_trie():
#     1:train 2:+vaild 3:+valid+test 4:3 + input
    str_seq = train_str_seq + input_str_seq + train_shift_str_seq + input_shift_str_seq     
    trie = marisa_trie.Trie(str_seq)
    count_signature = np.zeros(len(trie.items()), dtype = int)
    for i in range(len(str_seq)):
        count_signature[trie.key_id(unicode(str_seq[i]))] += 1
    return trie,count_signature

In [27]:
trie,count_signature = Construct_trie()
print(len(trie.items()),count_signature[:10])

(8331578, array([60,  2,  3,  7,  4,  1,  4,  1,  3,  2]))


Here are functions to check recurrence relations. This code is from Nina Chen.

https://www.kaggle.com/ncchen/integer-sequence-learning/recurrence-relation/discussion

In [28]:
def checkRecurrence(seq, order= 2, minlength = 5):
    """
    :type seq: List[int]
    :type order: int
    :type minlength: int 
    :rtype: List[int]
    
    Check whether the input sequence is a recurrence sequence with given order.
    If it is, return the coefficients for the recurrenec relation.
    If not, return None.
    """     
    if len(seq)< max((2*order+1), minlength):
        return None
    
    ################ Set up the system of equations 
    A,b = [], []
    for i in range(order):# style: use list to append and then transform to numpy
        A.append(seq[i:i+order])
        b.append(seq[i+order]) 
    A,b =np.array(A), np.array(b)
    try: 
        if np.linalg.det(A)==0: #no inverse
            return None
    except TypeError:
        return None
   
    #############  Solve for the coefficients (c0, c1, c2, ...)
    coeffs = np.linalg.inv(A).dot(b)  # x = inv(A) dot b
    
    ############  Check if the next terms satisfy recurrence relation
    for i in range(2*order, len(seq)):
        predict = np.sum(coeffs*np.array(seq[i-order:i]))
        if abs(predict-seq[i])>10**(-2): # if error < 0.01 return None.
            return None
    
    return list(coeffs)


def predictNextTerm(seq, coeffs):
    """
    :type seq: List[int]
    :type coeffs: List[int]
    :rtype: int
    
    Given a sequence and coefficienes, compute the next term for the sequence.
    """
    
    order = len(coeffs)
    predict = np.sum(coeffs*np.array(seq[-order:]))
    return int(round(predict))


In [15]:
seq = [1, 1, 2, 3, 5, 8, 13]
coeffs = checkRecurrence(seq, 2)
pred = predictNextTerm(seq, coeffs)
print(coeffs)
print(pred)

[1.0, 1.0]
21


Here are others function. 

In [29]:
import random
def checkGP(seq):
    if(len(seq) <= 2):
        return None
    div_seq = []
    for i in xrange(len(seq)-1):
        if seq[i] == 0:
            return None
        else:
            if seq[i+1] % seq[i] != 0:
                return None
            div_seq.append(seq[i+1] / float(seq[i]))
    if np.all(np.diff(div_seq,1) == 0):
        return int(seq[-1] * (seq[-1] / float(seq[-2]))) 
    else:
        return None

def findDerivative(seq):
    return [0] if len(seq)<=1 else [seq[i]-seq[i-1] for i in range(1,len(seq))]

def Newton_diff(seq):
    tail = []
    for i in range(len(seq)-1):
        if len(set(seq)) == 1:
            return sum(tail) + seq[-1]
        tail.append(seq[-1])
        seq = findDerivative(seq)
    return None

def math_diff(seq):
    pred = Newton_diff(seq)
    if pred != None:
        return pred
    pred = checkGP(seq)
    if pred != None:
        return pred
    for order in range(2,(len(seq)+1)/2):
        coeffs = checkRecurrence(seq, order)
        if coeffs != None:
            pred = predictNextTerm(seq, coeffs)
            break
    return pred

def trie_diff(seq):
    random.seed(4)
    shift = 3 
    boundary = 3
#     shift until length = 3
    pred = None
    tail = []
    pred_candidate = []
    for count_diff in range(1): 
        for i in range(len(seq)-boundary+1):
            shift_seq = seq[i:]
            signature, gcd = findSignature(shift_seq) 
            key = unicode(','.join(map(str,signature)) + ',')
            prefixes_in_trie = trie.keys(key)
            if prefixes_in_trie != []:
                start_index = len(key)
                count_prefix = [count_signature[trie.key_id(x)] for x in prefixes_in_trie]
                index = 0
                _ = np.max(count_prefix)
                index_cand = [x for x in range(len(count_prefix)) if count_prefix[x] == _]
                index = random.choice(index_cand)
                end_index = prefixes_in_trie[index].find(',',start_index)
                if end_index == -1:
                    end_index = None
                pred_str = prefixes_in_trie[index][start_index:end_index]
                if pred_str != '':
                    pred = int(pred_str) * gcd
                    pred_candidate.append((pred+sum(tail),len(shift_seq),count_diff))
        tail.append(seq[-1])
        seq = findDerivative(seq)
    pred_candidate = sorted(pred_candidate, key=lambda x: (-x[1],x[2]))
#     print(pred_candidate)
    if pred_candidate != []:
        return pred_candidate[0][0]
    return None
def pred_math_trie(seqs):
    count = 0
    seqs_pred, record = np.zeros(len(seqs),dtype = object), np.empty(len(seqs),dtype = (str,10))
    for i in xrange(len(seqs)):
        flag = 'math'
        pred = math_diff(seqs[i])
        if pred == None:
            pred = trie_diff(seqs[i])
            flag = 'trie'
        if pred != None:
            seqs_pred[i] = pred
            count += 1
            record[i] = flag
            print(i,pred,count,flag)
    return seqs_pred, record
_ = [1,-2,3,-4,5]
print(math_diff(_))
print(trie_diff(_))

-6
-6


run.

In [ ]:
pred ,record = pred_math_trie(input_seq)
print(sum(np.logical_or(record == 'trie', record == 'math')))

In [31]:
input_datadf.rename(columns={'Sequence': 'Last'}, inplace = True)
input_datadf['Last'] = pd.Series(pred)
input_datadf.set_index('Id',inplace = True)
input_datadf.to_csv('submit4.csv')

In [32]:
np_input_seq = np.array(input_seq)

3gram method. (experiment)

In [33]:
ngram_3 = []
for i in xrange(len(train_seq)):
    for j in range(len(train_seq[i])-2):
        ngram_3.append([train_seq[i][j], train_seq[i][j+1], train_seq[i][j+2]])
for i in xrange(len(input_seq)):
    for j in range(len(input_seq[i])-2):
        ngram_3.append([input_seq[i][j], input_seq[i][j+1], input_seq[i][j+2]])
print (ngram_3[:5])

[[1L, 3L, 13L], [3L, 13L, 87L], [13L, 87L, 1053L], [87L, 1053L, 28576L], [1053L, 28576L, 2141733L]]


In [34]:
print(len(ngram_3))

8903424


In [35]:
ngram_3_str_seq = []
for i in xrange(len(ngram_3)):
    signature, _ = findSignature(ngram_3[i])
    ngram_3_str_seq.append(','.join(map(str, signature)))
trie_ngram_3 = marisa_trie.Trie(ngram_3_str_seq)
count_signature_ngram_3 = np.zeros(len(trie_ngram_3.items()), dtype = int)
for i in xrange(len(ngram_3_str_seq)):
    count_signature_ngram_3[trie_ngram_3.key_id(unicode(ngram_3_str_seq[i]))] += 1
print(len(trie_ngram_3.items()))
print(trie_ngram_3.restore_key(0))
print(count_signature_ngram_3[:5])

3050248
0,0,0
[180647  85836  77537  50462  11975]


In [ ]:
pred_ngram_3 = np.zeros(len(input_seq),dtype = object)
pred_o = None
random.seed(4)
count = 0
for i in xrange(len(input_seq)):
    if record[i] != 'trie' and record[i] != 'math':
        seq = input_seq[i][-2:]
        signature, gcd = findSignature(seq) 
        key = unicode(','.join(map(str,signature)) + ',')
        ngram_3_in_trie = trie_ngram_3.keys(key)
        if ngram_3_in_trie != []:
            start_index = len(key)
            count_ngram_3 = [count_signature_ngram_3[trie_ngram_3.key_id(x)] for x in ngram_3_in_trie]
            _ = np.max(count_ngram_3)
            index_cand = [x for x in xrange(len(count_ngram_3)) if count_ngram_3[x] == _]
            index = random.choice(index_cand)
            end_index = ngram_3_in_trie[index].find(',',start_index)
            if end_index == -1:
                end_index = None
            pred_str = ngram_3_in_trie[index][start_index:end_index]
            if pred_str != '':
                pred_o = int(pred_str) * gcd
                count += 1
                record[i] = 'ngram_3'
                pred_ngram_3[i] = pred_o
                print(i,pred_o,count)